In [ ]:
import numpy as np
from pymatgen import Structure
from pymatgen.io.vasp.inputs import Poscar
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from scipy.optimize import minimize
import pandas as pd
import itertools, time
import argparse
import matplotlib.pyplot as plt
import pymatgen as mg
import seaborn as sns
import matplotlib.colors as mcolors
import sys
import re

def isanion(atom, anions=['O', 'S', 'F', 'Cl', 'N','Br']):
    #print "in isanion fun... atom is {} and anions are {}".format(atom, anions)
    check = atom in anions
    return check

def iscation(atom, cations=[]):
    check = atom not in ['O', 'S', 'F', 'Cl', 'N','Br'] 
    return check  

bv = pd.read_csv("Bond_valences2016_includes_lower_oxidation_states.csv")
bv.head()# Use element names and valences to lookup bond valence


def get_bv_params(cation, anion, cat_val, an_val):
    bond_val_list = bv[(bv['Atom1'] == cation) & (bv['Atom1_valence'] == cat_val)\
                  & (bv['Atom2'] == anion) & (bv['Atom2_valence'] == -1*an_val)]
    val = bond_val_list[bond_val_list['B'] == 0.37] #takes the bond parameters with B = 0.37
    if val.empty: # if B=0.37 does not exist for that pair of atoms, take the first value
        val = bond_val_list
    return val


def dict_bv_params(struct,formal_vals={}):
    bv_params_Ro = {}
    bv_params_B = {}

    long_elems = [str(i) for i in struct.species]
    elements = list(set(long_elems))
    
    # Creates empty dictionaries with the cations as keys
    for k in elements:
        if iscation(k):
            bv_params_Ro.setdefault(k,{})
            bv_params_B.setdefault(k,{})

    for i in elements:
        for j in elements:
            if iscation(j) and isanion(i):
                params = get_bv_params(cation=j, anion=i,
                                      cat_val = float(formal_vals[j]),
                                      an_val = float(formal_vals[i]))
                bv_params_Ro[j].update({i:params.iloc[0]['Ro']})
                bv_params_B[j].update({i:params.iloc[0]['B']})
    return bv_params_Ro, bv_params_B


# GII calculator for minimization of lattice parameters
# N.B. bond valence parameters must be generated before calling this function
def gii_fun_opt(x, *args):
    
    species_list = args[0]
    coords = args[1]
    formal_vals = args[2]
    params_Ro = args[3]
    params_B = args[4]
    
    struct = Structure(np.diag(x), species_list, coords)
    pymat_neighbors = struct.get_all_neighbors(6, include_index=True)
    values_BV = []
    
    for atom_indx, neigh_data in enumerate(pymat_neighbors):
        bv = 0
        for pair_data in neigh_data:
            atom = struct.species[atom_indx].symbol
            neighbor = struct.species[pair_data[2]].symbol
            if iscation(atom) and isanion(neighbor):
                bv += np.exp(((params_Ro[atom][neighbor]-pair_data[1])/params_B[atom][neighbor]))
            elif iscation(neighbor) and isanion(atom):
                bv += np.exp(((params_Ro[neighbor][atom]-pair_data[1])/params_B[neighbor][atom]))
        values_BV.append((formal_vals[struct.species[atom_indx].symbol] - bv)**2)

    GII_val = np.sqrt((sum(values_BV[:]))/struct.composition.num_atoms)
    return GII_val

# Calculate GII for a given structure
def gii_fun(struct, formal_vals={}, params_Ro={}, params_B={}):

    pymat_neighbors = struct.get_all_neighbors(6, include_index=True)
    values_BV = []
    
    for atom_indx, neigh_data in enumerate(pymat_neighbors):
        bv = 0
        for pair_data in neigh_data:
            atom = struct.species[atom_indx].symbol
            neighbor = struct.species[pair_data[2]].symbol
            if iscation(atom) and isanion(neighbor):
                bv += np.exp(((params_Ro[atom][neighbor]-pair_data[1])/params_B[atom][neighbor]))
            elif iscation(neighbor) and isanion(atom):
                bv += np.exp(((params_Ro[neighbor][atom]-pair_data[1])/params_B[neighbor][atom]))
        values_BV.append((formal_vals[struct.species[atom_indx].symbol] - bv)**2)

    GII_val = np.sqrt((sum(values_BV[:]))/struct.composition.num_atoms)
    return GII_val

site_IDs = pd.read_excel("Site_IDs.xlsx")

# Generates all charge-balanced compounds with a formula of the form A_{nA}B_{nB}O_{nO}X_{nX}
# Elements in A,B,X can be changed by changing the Site_IDs.xlsx sheet
def generate_formulas(nA, nB, nO, nX, X_anion):
    
    A = dict(site_IDs['AsiteID'])
    B = dict(site_IDs['BsiteID'])
    X = dict(site_IDs['XsiteID'])
    
    elems = []
    unrounded_vals = []
    anion = []
    rounded_vals = []
    elems_x = []
    formal_vals = {}
    
    for i in A:
        for j in B:
            for k in X:
                if (site_IDs.loc[i,'Avalence']*nA + site_IDs.loc[j,'Bvalence']*nB - 2*nO
                    -site_IDs.loc[k,'Xvalence']) == 0:
                    elems.append([site_IDs.loc[i,'ElemA'], site_IDs.loc[j,'ElemB'],'O',site_IDs.loc[k,'ElemX']])
                    unrounded_vals.append([site_IDs.loc[i,'Avalence'], site_IDs.loc[j,'Bvalence'],
                                           2, site_IDs.loc[k,'Xvalence']])
    rounded_vals = np.around(unrounded_vals)
    
    for i in range(len(elems)):
        if elems[i][3] == X_anion:
            elems_x.append(elems[i])
            formal_vals.update(dict(zip(elems[i],rounded_vals[i])))

    return (elems_x, formal_vals)

# Imposes a penalty if orthogonal cells become far from tetragonal
def check_tetragonality(x, *args):
    
    species_list = args[0]
    coords = args[1]
    formal_vals = args[2]
    params_Ro = args[3]
    params_B = args[4]
    
    struct = Structure(np.diag(x), species_list, coords)
    
    if abs(struct.lattice.abc[0]-struct.lattice.abc[2]) or abs(struct.lattice.abc[0]-struct.lattice.abc[1]) < 0.01:
        val = 0
    else: 
        val = abs(struct.lattice.abc[0]-struct.lattice.abc[2])*10
    
    return val
    
def minime(x, *args):
    return gii_fun_opt(x,*args) + check_tetragonality(x,*args)

In [ ]:
# GII minimizing routine. This will generate the GIIs of structures before and after the lattice relaxation.

gii_orig =[]
gii_opt = []
bad_relax = [] # Removes compounds whose GIIs are outside the bounds set for the lattice parameters
bnds = ((3,6),(3,6),(11.5,14.5)) # Set bounds for lattice parameters
elems, formal_vals = generate_formulas(2,1,3,1,'F')

for i in range(len(elems)):

    struct = mg.Structure.from_file("M3-_a_P4nmm.cif") # Change input structure here, must have A = Sr, B = Ti, and X=F
    struct['Sr'] = elems[i][0]
    struct['Ti'] = elems[i][1]
    struct['O'] = elems[i][2]
    struct['F'] = elems[i][3]
    temp_form = struct.composition.reduced_formula
    params_Ro, params_B = dict_bv_params(struct,formal_vals)
    
    # obtain baseline values for GII 
    temp_gii = gii_fun(struct,formal_vals,params_Ro,params_B)
    gii_orig.append([temp_form, temp_gii])
    
    # Optimize the lattice constants using GII minimizer
    coords = struct.frac_coords
    x0 = struct.lattice.abc
    species = struct.species
    myargs = (species, coords, formal_vals, params_Ro, params_B)
    res = minimize(minime, x0, myargs, method='SLSQP', bounds=bnds, options={'disp':True})
    gii_opt.append(res.fun)
    opt_struct = Structure(np.diag(res.x),species,coords)
    w = Poscar(opt_struct)
    w.write_file("{}_opt_structure.vasp".format(temp_form)) # Outputs structures to .vasp file
    if res.x[0]==3.0 or res.x[1]==3.0 or res.x[2]==11.5 or res.x[0]==6.0 or res.x[2]==6.0 or res.x[1]==14.5:
        bad_relax.append(temp_form)

df_GIIs = pd.DataFrame(gii_orig, columns = ['formula', 'original'])
df_GIIs = df_GIIs.assign(opt=gii_opt)


In [ ]:
# Plot heatmap of original GII and optimized GII
%matplotlib inline
df_temp = df_GIIs.set_index('formula')
df_transpose_GIIs = df_temp.transpose()

# Only plot compounds which have GIIs less than 0.6 
df_transpose_GIIS = df_transpose_GIIs[df_transpose_GIIs.columns[0.6 > df_transpose_GIIs.min()]]

# get rid of any formulas whose lattice relaxations diverge
for i in bad_relax:
    if i in df_transpose_GIIs:
        df_transpose_GIIs.pop(i)

fig,ax = plt.subplots()
fig.set_size_inches(20,3)

fig = sns.heatmap(df_transpose_GIIs, annot=True, linewidth=0.25, vmax=0.6, vmin=0)
#plt.savefig('GII_heatmap.png',dpi=300,bbox_inches='tight')